In [37]:
from bs4 import BeautifulSoup as bs
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import steam_review_scraper 
import requests
from datetime import datetime as dt
import pandas as pd
from steam_review_scraper import get_game_review 
import pickle
import steamreviews
import re
import warnings

In [ ]:
#https://steamcommunity.com/app/1517290/reviews/?browsefilter=mostrecent&snr=1_5_100010_&p=1 steam website
driver = webdriver.Chrome("chromedriver.exe") 
driver.get("https://steamcommunity.com/app/1517290/reviews/?browsefilter=mostrecent&snr=1_5_100010_&p=1")#navgiate to website
time.sleep(1)
element=driver.find_element(By.CLASS_NAME,"btn_grey_white_innerfade") # find element
element.click() #click on element

In [ ]:
#open drop down list for review ratings
element= driver.find_element(By.CLASS_NAME, "user_reviews_rating_filter")
element.click()

In [ ]:
#click on positive reviews only
driver.find_element(By.ID,"filterrating_option_1").click()

In [ ]:
pre_scroll_height = driver.execute_script('return document.body.scrollHeight;')
run_time, max_run_time = 0, 2 

while True:
    iteration_start = time.time() #start time
    driver.execute_script('window.scrollTo(0, 100*document.body.scrollHeight);') # scroll to the end of the page

    post_scroll_height = driver.execute_script('return document.body.scrollHeight;') #retrun the height of the page after scrolling

    scrolled = post_scroll_height != pre_scroll_height #to check if post scroll and pre scroll height are the same
    timed_out = run_time >= max_run_time #time does not go over 2 seconds when loading page

    if scrolled:
        run_time = 0 
        pre_scroll_height = post_scroll_height
    elif not scrolled and not timed_out:
         run_time += time.time() - iteration_start
    elif not scrolled and timed_out:
         break
    try:
        element = driver.find_element(By.ID,"GetMoreContentBtn")
        element.click()
    except:
        continue

In [ ]:
#scrape date from loaded website
soup = bs(driver.page_source, "lxml")

In [ ]:
#write the website content to a local file
#with open("output1.html", "w", encoding='utf-8') as file:
#    file.write(str(soup))

In [ ]:
#open drop down list for review ratings
element= driver.find_element(By.CLASS_NAME, "user_reviews_rating_filter")
element.click()

In [ ]:
#click on negative reviews only
driver.find_element(By.ID,"filterrating_option_2").click()

In [ ]:
pre_scroll_height = driver.execute_script('return document.body.scrollHeight;')
run_time, max_run_time = 0, 2 

while True:
    iteration_start = time.time() #start time
    driver.execute_script('window.scrollTo(0, 100*document.body.scrollHeight);') # scroll to the end of the page

    post_scroll_height = driver.execute_script('return document.body.scrollHeight;') #retrun the height of the page after scrolling

    scrolled = post_scroll_height != pre_scroll_height #to check if post scroll and pre scroll height are the same
    timed_out = run_time >= max_run_time #time does not go over 2 seconds when loading page

    if scrolled:
        run_time = 0 
        pre_scroll_height = post_scroll_height
    elif not scrolled and not timed_out:
         run_time += time.time() - iteration_start
    elif not scrolled and timed_out:
         break
    try:
        element = driver.find_element(By.ID,"GetMoreContentBtn")
        element.click()
    except:
        continue

In [ ]:
#scrape date from loaded website
soup = bs(driver.page_source, "lxml")

In [ ]:
#write the website content to a local file
#with open("output1.html", "w", encoding='utf-8') as file:
#   file.write(str(soup))

In [42]:
#with open("output2.html", "w", encoding='utf-8') as file:
#    file.write(str(soup))

In [3]:
#open local hmtl file which contains all postive reviews
HTMLFileToBeOpened = open("output1.html", "r",encoding="utf8")
contents = HTMLFileToBeOpened.read()
beautifulSoupText = bs(contents, 'lxml')

In [21]:
columns = ["helpful","recommendations","hours","review","date"]
BFReivews=pd.DataFrame(columns=columns)
temp = pd.DataFrame(columns =columns,index =range(1))

In [33]:
numberofPostiveReviews=len(beautifulSoupText.find_all("div", {"class": "apphub_UserReviewCardContent"}))
allpositiveReviews=beautifulSoupText.find_all("div", {"class": "apphub_UserReviewCardContent"})
warnings.filterwarnings('ignore')

for i in range(numberofPostiveReviews):
    
    userReview = allpositiveReviews[i]
    
    helpful =userReview.find("div", {"class": "found_helpful"})
    helpful = re.sub("<.*?>","",str(helpful))
    helpful = re.sub("\t","",helpful)
    helpful = re.sub("\n","",helpful)
    
    
    recommendations =userReview.find("div", {"class": "title"})
    recommendations =re.sub("<.*?>","",str(recommendations))
    
    hours =userReview.find("div",{"class":"hours"})
    hours = re.sub("<.*?>","",str(hours))
    
    review = userReview.find("div", {"class": "apphub_CardTextContent"})
    review = re.sub("<.*>","",str(review))
    review = re.sub("\n","",review)
    reivew = re.sub("\t","",review)
    
    date = userReview.find("div",{"class":"date_posted"})
    date = re.sub("<.*?>","",str(date))
    
    temp["helpful"]= helpful
    temp["recommendations"] = recommendations
    temp["hours"] =hours
    temp["review"] = review
    temp["date"] = date
    
    BFReivews = BFReivews.append(temp,ignore_index = True)
    temp = temp.iloc[0:0]
    temp=temp.append(pd.Series(), ignore_index=True)

In [38]:
#open local file
HTMLFileToBeOpened = open("output2.html", "r",encoding="utf8")
contents = HTMLFileToBeOpened.read()
beautifulSoupText = bs(contents, 'lxml')

In [39]:
numberofNegativeReviews=len(beautifulSoupText.find_all("div", {"class": "apphub_UserReviewCardContent"}))
allNegativeReviews=beautifulSoupText.find_all("div", {"class": "apphub_UserReviewCardContent"})

for i in range(numberofNegativeReviews):
    
    userReview = allNegativeReviews[i]
    
    helpful =userReview.find("div", {"class": "found_helpful"})
    helpful = re.sub("<.*?>","",str(helpful))
    helpful = re.sub("\t","",helpful)
    helpful = re.sub("\n","",helpful)
    
    
    recommendations =userReview.find("div", {"class": "title"})
    recommendations =re.sub("<.*?>","",str(recommendations))
    
    hours =userReview.find("div",{"class":"hours"})
    hours = re.sub("<.*?>","",str(hours))
    
    review = userReview.find("div", {"class": "apphub_CardTextContent"})
    review = re.sub("<.*>","",str(review))
    review = re.sub("\n","",review)
    reivew = re.sub("\t","",review)
    
    date = userReview.find("div",{"class":"date_posted"})
    date = re.sub("<.*?>","",str(date))
    
    temp["helpful"]= helpful
    temp["recommendations"] = recommendations
    temp["hours"] =hours
    temp["review"] = review
    temp["date"] = date
    
    BFReivews = BFReivews.append(temp,ignore_index = True)
    temp = temp.iloc[0:0]
    temp=temp.append(pd.Series(), ignore_index=True)

In [41]:
BFReivews.to_csv("Battlefieldreviews.csv")